In [1]:
from flask import Flask, render_template, request,jsonify
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq

In [2]:
searchString = "Narendra Modi"

In [46]:
def fetch_image_urls(query: str, max_links_to_fetch: int, sleep_between_interactions: int = 1):

    # build the flipkart query
    search_url = "https://www.flipkart.com/search?q=*{0}"
    
    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        # get all image thumbnail results
        uClient = uReq(search_url.format(query))
        flipkartPage = uClient.read()
        uClient.close()
        search_page_html = bs(flipkartPage, "html.parser")
        thumbnail_results = search_page_html.findAll("img", {"class": "_396cs4"})
        number_results = len(thumbnail_results)

        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        print(type(thumbnail_results))
        print(extract_image_urls(thumbnail_results, results_start, number_results))
        image_urls = image_urls.union(extract_image_urls(thumbnail_results, results_start, number_results))
                
        image_count = len(image_urls)
        print(image_count)
        print(image_urls)
        # Extract more images to meet the count
#         if len(image_urls) >= max_links_to_fetch:
#             print(f"Found: {len(image_urls)} image links, done!")
#             break
#         else:
#             print("Found:", len(image_urls), "image links, looking for more ...")
#             time.sleep(30)
#             return
#             load_more_button = wd.find_element_by_css_selector("._1LKTO3")
#             if load_more_button:
#                 wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
#         results_start = len(thumbnail_results)

    return image_urls

In [44]:
def extract_image_urls(thumbnail_results: list, results_start: int, number_results: int):
    image_urls = set()
    try:
        for img in thumbnail_results[results_start:number_results]:
            # extract image urls
            print(thumbnail_results)
            print(img)
            print(img['src'])
            if img['src'] and 'http' in img['src']:
                image_urls.add(img['src'])
                
    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
    print(image_urls)    
    return image_urls

In [5]:
def persist_image(folder_path:str,url:str, counter):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb')
        f.write(image_content)
        f.close()
        print(f"SUCCESS - saved {url} - as {folder_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [6]:
from selenium import webdriver
import os
def search_and_download(search_term: str, driver_path: str, target_path='./images', number_images=10):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, sleep_between_interactions=0.5)

    counter = 0
    for elem in res:
        persist_image(target_folder, elem, counter)
        counter += 1

In [48]:
DRIVER_PATH = r'./chromedriver.exe'
search_term = 'Iphone13'
# num of images you can pass it from here  by default it's 10 if you are not passing
#number_images = 50
search_and_download(search_term=search_term, driver_path=DRIVER_PATH, number_images=10)

C:\Users\ashwin.prabhu\AppData\Local\Temp\ipykernel_16512\1606131757.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:


Found: 24 search results. Extracting links from 0:24
<class 'bs4.element.ResultSet'>
[<img alt="APPLE iPhone 13 (Starlight, 128 GB)" class="_396cs4 _3exPp9" src="https://rukminim1.flixcart.com/image/312/312/ktketu80/mobile/6/n/d/iphone-13-mlpg3hn-a-apple-original-imag6vpyghayhhrh.jpeg?q=70"/>, <img alt="APPLE iPhone 13 (Green, 128 GB)" class="_396cs4 _3exPp9" src="https://rukminim1.flixcart.com/image/312/312/l0igvww0/mobile/r/q/m/-original-imagca5ajerqpfjy.jpeg?q=70"/>, <img alt="APPLE iPhone 13 (Midnight, 128 GB)" class="_396cs4 _3exPp9" src="https://rukminim1.flixcart.com/image/312/312/ktketu80/mobile/s/l/c/iphone-13-mlpf3hn-a-apple-original-imag6vzz5qvejz8z.jpeg?q=70"/>, <img alt="APPLE iPhone 13 (Blue, 128 GB)" class="_396cs4 _3exPp9" src="https://rukminim1.flixcart.com/image/312/312/ktketu80/mobile/2/y/o/iphone-13-mlpk3hn-a-apple-original-imag6vpyur6hjngg.jpeg?q=70"/>, <img alt="APPLE iPhone 13 (Midnight, 256 GB)" class="_396cs4 _3exPp9" src="https://rukminim1.flixcart.com/image/3

SUCCESS - saved https://rukminim1.flixcart.com/image/312/312/l0igvww0/mobile/r/q/m/-original-imagca5ajerqpfjy.jpeg?q=70 - as ./images\iphone13
SUCCESS - saved https://rukminim1.flixcart.com/image/312/312/ktketu80/mobile/6/n/d/iphone-13-mlpg3hn-a-apple-original-imag6vpyghayhhrh.jpeg?q=70 - as ./images\iphone13
SUCCESS - saved https://rukminim1.flixcart.com/image/312/312/k2jbyq80pkrrdj/mobile-refurbished/u/z/t/iphone-11-256-c-mwmd2hn-a-apple-0-original-imafkg24wszdhzy3.jpeg?q=70 - as ./images\iphone13
SUCCESS - saved https://rukminim1.flixcart.com/image/312/312/ktketu80/mobile/a/m/7/iphone-13-mlpj3hn-a-apple-original-imag6vpyk3w4zarg.jpeg?q=70 - as ./images\iphone13
SUCCESS - saved https://rukminim1.flixcart.com/image/312/312/ktketu80/mobile/s/l/c/iphone-13-mlpf3hn-a-apple-original-imag6vzz5qvejz8z.jpeg?q=70 - as ./images\iphone13
SUCCESS - saved https://rukminim1.flixcart.com/image/312/312/ktketu80/mobile/8/z/w/iphone-13-mlph3hn-a-apple-original-imag6vzzhrxgazsg.jpeg?q=70 - as ./images\